Natural Language processing Assignment-05

Implement a Hidden Markov Model (HMM) for weather forecasting, taking scenario where we have hidden weather states (e.g., "Sunny" and "Rainy") and observable events (e.g., "Dry", "Damp", "Wet"). The goal is to predict the sequence of weather states based on the observations. [CO2]

In [6]:
!pip install hmmlearn

In [3]:
import numpy as np
from hmmlearn import hmm
states = ["Sunny", "Rainy"]
n_states = len(states)
observations = ["Dry", "Damp", "Wet"]
n_observations = len(observations)
transition_prob = np.array([
    [0.7, 0.3],  # Sunny -> Sunny, Sunny -> Rainy
    [0.4, 0.6]   # Rainy -> Sunny, Rainy -> Rainy
])

Problem Setup
States (S): Hidden states representing the weather (e.g., "Sunny", "Rainy").
Observations (O): Observable events that we can measure (e.g., "Dry", "Damp", "Wet").
Transition Probabilities (A): The probability of transitioning from one weather state to another.
Emission Probabilities (B): The probability of observing a certain event given the weather state.
Initial Probabilities (π): The probability distribution over the initial states.

In [4]:
import numpy as np
states = ["Sunny", "Rainy"]
observations = ["Dry", "Damp", "Wet"]
transition_prob = np.array([[0.8, 0.2],  # P(Sunny -> Sunny), P(Sunny -> Rainy)
                            [0.4, 0.6]]) # P(Rainy -> Sunny), P(Rainy -> Rainy)
emission_prob = np.array([[0.6, 0.3, 0.1],  # P(Dry|Sunny), P(Damp|Sunny), P(Wet|Sunny)
                          [0.1, 0.4, 0.5]]) # P(Dry|Rainy), P(Damp|Rainy), P(Wet|Rainy)
initial_prob = np.array([0.5, 0.5])  # P(Sunny), P(Rainy)
obs_seq = [0, 1, 2]  # ["Dry", "Damp", "Wet"]
def viterbi(obs_seq, states, initial_prob, transition_prob, emission_prob):
    n_states = len(states)
    T = len(obs_seq)
    dp = np.zeros((n_states, T))
    path = np.zeros((n_states, T), dtype=int)
    for s in range(n_states):
        dp[s, 0] = initial_prob[s] * emission_prob[s, obs_seq[0]]
    for t in range(1, T):
        for s in range(n_states):
            probabilities = dp[:, t-1] * transition_prob[:, s] * emission_prob[s, obs_seq[t]]
            dp[s, t] = np.max(probabilities)
            path[s, t] = np.argmax(probabilities)
    # Backtrack to find the optimal state sequence
    optimal_path = np.zeros(T, dtype=int)
    optimal_path[T-1] = np.argmax(dp[:, T-1])
    for t in range(T-2, -1, -1):
        optimal_path[t] = path[optimal_path[t+1], t+1]
    optimal_states = [states[state] for state in optimal_path]
    return optimal_states
# Predict the most likely sequence of states
predicted_states = viterbi(obs_seq, states, initial_prob, transition_prob, emission_prob)
print("Most likely weather sequence:", predicted_states)

Most likely weather sequence: ['Sunny', 'Sunny', 'Rainy']


Take different transition and emission probabilities or observation sequences on above problem and see how the model's predictions change. [CO2]

In [5]:
# Emission probabilities (B)
emission_prob = np.array([
    [0.8, 0.15, 0.05],  # Sunny -> Dry, Damp, Wet
    [0.1, 0.35, 0.55]   # Rainy -> Dry, Damp, Wet
])
# Initial probabilities (π)
start_prob = np.array([0.6, 0.4])  # Sunny, Rainy
# Create HMM model
model = hmm.MultinomialHMM(n_components=n_states)
model.startprob_ = start_prob
model.transmat_ = transition_prob
model.emissionprob_ = emission_prob
observation_sequence = np.array([0, 1, 2, 0, 1]).reshape(-1, 1)  # Example observation sequence
print("Observation sequence:", [observations[i[0]] for i in observation_sequence])
new_observation_sequence = np.array([2, 2, 1, 0]).reshape(-1, 1)  # Example: Wet -> Wet -> Damp -> Dry
#logprob_new, hidden_states_new = model.decode(new_observation_sequence, algorithm="viterbi")
print("New Observation sequence:", [observations[i[0]] for i in new_observation_sequence])

https://github.com/hmmlearn/hmmlearn/issues/335
https://github.com/hmmlearn/hmmlearn/issues/340


Observation sequence: ['Dry', 'Damp', 'Wet', 'Dry', 'Damp']
New Observation sequence: ['Wet', 'Wet', 'Damp', 'Dry']


Take different transition and emission probabilities or observation sequences on above problem and see how the model's predictions change. [CO2]

In [8]:
import numpy as np
from hmmlearn import hmm

# Function to predict hidden states using HMM with GaussianHMM
def run_hmm(obs_sequence, transition_prob, emission_prob, start_prob):
    states = ["Sunny", "Rainy"]
    n_states = len(states)

    observations = ["Dry", "Damp", "Wet"]
    obs_map = {"Dry": 0, "Damp": 1, "Wet": 2}

    # Convert observation sequence into indices
    obs_sequence = np.array([[obs_map[obs]] for obs in obs_sequence])

    # Initialize the HMM model (GaussianHMM for categorical observation handling)
    model = hmm.GaussianHMM(n_components=n_states, covariance_type="full", n_iter=100)

    # Set model parameters manually
    model.startprob_ = start_prob
    model.transmat_ = transition_prob

    # Emission means and covariances for each state
    means = np.array([[0], [1]])  # Mean of observation Dry/Damp/Wet for each state
    covars = np.array([[[0.1]], [[0.1]]])  # Small variance to represent categorical nature

    model.means_ = means
    model.covars_ = covars

    # Predict the hidden states using the Viterbi algorithm
    logprob, state_sequence = model.decode(obs_sequence, algorithm="viterbi")
    predicted_states = [states[i] for i in state_sequence]

    return predicted_states

# Test with different configurations of transition and emission probabilities

# Configuration 1: Transition and Emission probabilities
transition_prob_1 = np.array([
    [0.8, 0.2],  # Sunny -> Sunny, Sunny -> Rainy
    [0.4, 0.6],  # Rainy -> Sunny, Rainy -> Rainy
])

emission_prob_1 = np.array([
    [0.7, 0.2, 0.1],  # Sunny: Dry, Damp, Wet
    [0.1, 0.3, 0.6],  # Rainy: Dry, Damp, Wet
])

start_prob_1 = np.array([0.6, 0.4])  # Initial probabilities for Sunny, Rainy

# Observation sequences for testing
obs_sequence_1 = ["Dry", "Damp", "Wet"]
obs_sequence_2 = ["Wet", "Damp", "Dry"]

# Run HMM for different configurations and observation sequences
predicted_states_1_config1 = run_hmm(obs_sequence_1, transition_prob_1, emission_prob_1, start_prob_1)
predicted_states_2_config1 = run_hmm(obs_sequence_2, transition_prob_1, emission_prob_1, start_prob_1)

# Print the results
print("Config 1 with obs_sequence_1:", predicted_states_1_config1)
print("Config 1 with obs_sequence_2:", predicted_states_2_config1)


Config 1 with obs_sequence_1: ['Sunny', 'Rainy', 'Rainy']
Config 1 with obs_sequence_2: ['Rainy', 'Rainy', 'Sunny']
